In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [ ]:
%%time
X_train = np.load("../data/array_data_mini/X_train.npz")["arr_0"]
y_train = np.load("../data/array_data_mini/y_train.npz")["arr_0"]
X_val = np.load("../data/array_data_mini/X_val.npz")["arr_0"]
y_val = np.load("../data/array_data_mini/y_val.npz")["arr_0"]

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

In [ ]:
seq_size = (X_train[0].shape[0],1)
n_classes = 6
batch_size = 32

In [ ]:
class Gen_Data(tf.keras.utils.Sequence):
    
    # generate data batches for model
    
    def __init__(self,batch_size,seq_size,X_data,y_data):
        
        # set required attributes
        self.batch_size = batch_size
        self.t_size = seq_size
        self.X_data = X_data
        self.y_data = y_data
        self.n_batches = len(self.X_data)//self.batch_size
        self.batch_idx = np.array_split(range(len(self.X_data)), self.n_batches)
            
    def __len__(self):

        # set number of batches per epoch
        return self.n_batches
    
    def __getitem__(self,idx):

        # fetch one batch   
        batch_X = self.X_data[self.batch_idx[idx]]
        batch_y = self.y_data[self.batch_idx[idx]]
        return batch_X, batch_y      

In [ ]:
train_gen = Gen_Data(batch_size, seq_size, X_train, y_train)
val_gen = Gen_Data(batch_size, seq_size, X_val, y_val)

In [ ]:
def get_model(seq_size, n_classes):
    inputs = tf.keras.Input(shape=seq_size)

    # entry block
    x = tf.keras.layers.Conv1D(32, 5, strides=2, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)
    block_out = x

    # 3 downsampling blocks    
    for f in [64,128,256]:
        
        x = tf.keras.layers.Conv1D(f, 5, strides=1, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)

        x = tf.keras.layers.Conv1D(f, 5, strides=1, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)

        x = tf.keras.layers.MaxPooling1D(3, strides=2, padding="same")(x)

        residual = tf.keras.layers.Conv1D(f, 1, strides=2, padding="same")(block_out)
        x = tf.keras.layers.add([x, residual])
        block_out = x   

    # 4 upsampling blocks        
    for f in [256, 128, 64, 32]:
        
        x = tf.keras.layers.Conv1DTranspose(f,5,padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x)        
        
        x = tf.keras.layers.Conv1DTranspose(f,5,padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation("relu")(x) 
        
        x = tf.keras.layers.UpSampling1D(2)(x) 
                
        residual = tf.keras.layers.UpSampling1D(2)(block_out)
        residual = tf.keras.layers.Conv1D(f, 1, padding="same")(residual)
        x = tf.keras.layers.add([x, residual])
        block_out = x      
                
    # pixel classification layer
    outputs = tf.keras.layers.Conv1D(n_classes, 3, activation="softmax", padding="same")(x)    
    
    # define the model
    model = tf.keras.Model(inputs, outputs)
    return model    

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model = get_model(seq_size,n_classes)

In [ ]:
model.summary()

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
opt = tf.keras.optimizers.Adam()

In [ ]:
model.compile(optimizer=opt,loss=loss)

In [ ]:
callbacks = [tf.keras.callbacks.ModelCheckpoint("unet_trial_1d.h5",monitor="val_loss",save_best_only=True)]

In [ ]:
epochs = 10
history = model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=callbacks)

In [ ]:
model.save()

In [ ]:
with open(f"history", "wb") as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
!tar cvfz allfiles.tar.gz *